In [ ]:
# PINN CUSTOM LOSS FUNCTION - V.P. Carey, ME249 F25

import tensorflow as tf
from tensorflow.keras import backend as K

def custom_loss_function(kappa, x_max, y_max, epoch_tracker):
    """ Custom loss function for the neural network model.
        - kappa: weight for the Laplace residual term.
        - x_max, y_max: maximum x and y coordinates of the domain.
        - epoch_tracker: callback object to keep track of the current epoch. """
    
    def loss(y_true, y_pred):    
        # Define pointsBoundaryChk as a constant tensor
        pointsBoundaryChk = tf.constant([
            [ 0.0, 0.0 ],
            [ 0.0, 0.5 ],
            [ 0.0, 1.0 ],
            [ 0.0, 1.5 ],
            [ 0.0, 2.0 ],
            [ 0.0, 2.5 ],
            [ 0.0, 3.0 ],
            [ 0.0, 3.5 ],
            [ 0.0, 4.0 ],
            [ 0.0, 4.5 ],
            [ 0.0, 5.0 ],
            [ 0.5, 5.0 ],
            [ 1.0, 5.0 ],
            [ 1.5, 5.0 ],
            [ 2.0, 5.0 ],
            [ 2.5, 5.0 ],
            [ 3.0, 5.0 ],
            [ 3.0, 4.5 ],
            [ 3.0, 4.0 ],
            [ 3.0, 3.5 ],
            [ 3.0, 3.0 ],
            [ 3.0, 2.5 ],
            [ 3.0, 2.0 ],
            [ 3.0, 1.5 ],
            [ 3.0, 1.0 ],
            [ 3.0, 0.5 ],
            [ 3.0, 0.0 ],
            [ 2.5, 0.0 ],
            [ 2.0, 0.0 ],
            [ 1.5, 0.0 ],
            [ 1.0, 0.0 ],
            [ 0.5, 0.0 ],
        ], dtype=tf.float32)
        
        # Boundary loss: Calculate squared error for the boundary condition
        boundary_loss = K.mean(K.square(y_pred - y_true))
        
        ESbdsum = 0.
        for _ in range(32):
            # Sample a random index and gather point
            idx = tf.random.uniform(shape=[], minval=0, maxval=32, dtype=tf.int32)
            selected_point = tf.gather(pointsBoundaryChk, idx)
                
            # Stack selected_point as input for the model (shape becomes (1, 2))
            input_tensor = tf.expand_dims(selected_point, axis=0)
            # Pass input_tensor as the input to the model
            T_pred_at_boundxy = model(input_tensor)  # Model prediction
            # add penalty if temperature at flux location is OUTSIDE RANGE 5-55 DEG C
            if (T_pred_at_boundxy < 5.):
                ESbdsum += 20.*(5. - T_pred_at_boundxy)*(5. - T_pred_at_boundxy)
            if (T_pred_at_boundxy > 55.):
                ESbdsum += 20.*(T_pred_at_boundxy - 55.)*(T_pred_at_boundxy - 55.)
        boundary_loss += ESbdsum/32.
    
        # Check epoch from the callback
        epoch = epoch_tracker.epochs_trained
            
        # If current epoch is less than N_epInit, ignore the Laplace residual
        if epoch < epoch_tracker.N_epInit:
            total_loss = boundary_loss
        else:
            # 3. Randomly select an interior point from this list using TensorFlow's random uniform
            points = tf.constant([
                [ 0.5, 0.5 ],
                [ 1.0, 0.5 ],
                [ 1.5, 0.5 ],
                [ 2.0, 0.5 ],
                [ 2.5, 0.5 ],
                [ 0.5, 1.0 ],
                [ 1.0, 1.0 ],
                [ 1.5, 1.0 ],
                [ 2.0, 1.0 ],
                [ 2.5, 1.0 ],
                [ 0.5, 1.5 ],
                [ 1.0, 1.5 ],
                [ 1.5, 1.5 ],
                [ 2.0, 1.5 ],
                [ 2.5, 1.5 ],
                [ 0.5, 2.0 ],
                [ 1.0, 2.0 ],
                [ 1.5, 2.0 ],
                [ 2.0, 2.0 ],
                [ 2.5, 2.0 ],
                [ 0.5, 2.5 ],
                [ 1.0, 2.5 ],
                [ 1.5, 2.5 ],
                [ 2.0, 2.5 ],
                [ 2.5, 2.5 ],
                [ 0.5, 3.0 ],
                [ 1.0, 3.0 ],
                [ 1.5, 3.0 ],
                [ 2.0, 3.0 ],
                [ 2.5, 3.0 ],
                [ 0.5, 3.5 ],
                [ 1.0, 3.5 ],
                [ 1.5, 3.5 ],
                [ 2.0, 3.5 ],
                [ 2.5, 3.5 ],
                [ 0.5, 4.0 ],
                [ 1.0, 4.0 ],
                [ 1.5, 4.0 ],
                [ 2.0, 4.0 ],
                [ 2.5, 4.0 ],
                [ 0.5, 4.5 ],
                [ 1.0, 4.5 ],
                [ 1.5, 4.5 ],
                [ 2.0, 4.5 ],
                [ 2.5, 4.5 ],
            ], dtype=tf.float32)
            
            meanLap_res = 0.
            meanTlimPen = 0.
            for _ in range(45):
                ##compute Laplacian residue here - need to do this once per BC point - include in range 32 loop =======
                # Use tf.random.uniform to sample an index for the interior point
                idx = tf.random.uniform(shape=[], minval=0, maxval=45, dtype=tf.int32)
                selected_point = tf.gather(points, idx)
                x_interior, y_interior = selected_point[0], selected_point[1],
                
                # Use tf.GradientTape to compute second derivatives
                with tf.GradientTape(persistent=True) as tape:
                    tape.watch(selected_point)
                    # Pass the selected (x_interior, y_interior) to the model to get T_pred_at_interior
                    T_pred_at_interior = model([x_interior, y_interior])  # Model prediction
                
                # Compute first derivatives with respect to x and y
                dT_dx = tape.gradient(T_pred_at_interior, selected_point[0])
                dT_dy = tape.gradient(T_pred_at_interior, selected_point[1])
                
                # Compute second derivatives (Hessian)
                d2T_dx2 = tape.gradient(dT_dx, selected_point[0])
                d2T_dy2 = tape.gradient(dT_dy, selected_point[1])
                
                # Calculate the Laplacian (sum of second derivatives)
                laplacian = d2T_dx2 + d2T_dy2
                
                # Compute the Laplacian residual (sum of squared Laplacian)
                meanLap_res +=  K.square(laplacian)/45.
                
                # ADD PENALTY IF T below 5 or above 55
            
                if (T_pred_at_interior < 5.):
                    meanTlimPen += 20.*(5. - T_pred_at_interior)*(5. - T_pred_at_interior)/45.
                if (T_pred_at_interior > 55.):
                    meanTlimPen += 20.*(T_pred_at_interior - 55.)*(T_pred_at_interior - 55.)/45.
                
            #Combine three losses
            total_loss = boundary_loss + kappa * meanLap_res + meanTlimPen

    
        return total_loss
    return loss
    